In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds

In [3]:
imdb_sentences = []
train_data = tfds.as_numpy(tfds.load('imdb_reviews', split="train"))
for item in train_data:
    imdb_sentences.append(str(item['text']))

Dataset imdb_reviews downloaded and prepared to C:\Users\Amanpreet Singh\tensorflow_datasets\imdb_reviews\plain_text\1.0.0. Subsequent calls will reuse this data.


In [9]:
print(type(train_data))
print(len(imdb_sentences))
for item in train_data:
    print(type(item))
    print(item.keys())
    print(item['label'])
    print(item['text'])
    #print(item)
    break

<class 'tensorflow_datasets.core.dataset_utils._IterableDataset'>
25000
<class 'dict'>
dict_keys(['label', 'text'])
0
b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it."
{'label': 0, 'text': b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be

In [10]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=5000)
tokenizer.fit_on_texts(imdb_sentences)
sequences = tokenizer.texts_to_sequences(imdb_sentences)

In [23]:
#print(tokenizer.word_index)

### starting writing cleaned code

In [25]:
!pip install beautifulsoup4

In [26]:
from bs4 import BeautifulSoup   #to remove html tags like <br> from the data 
import string    #to remove punctuation marks from the data

In [27]:
stopwords = ["a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "as", "at", "be", "because", "been", "before", "being", "below", "between", "both", "but", "by", "could", "did", "do", "does", "doing", "down", "during", "each", "few", "for", "from", "further", "had", "has", "have", "having", "he", "he'd", "he'll", "he's", "her", "here", "here's", "hers", "herself", "him", "himself", "his", "how", "how's", "i", "i'd", "i'll", "i'm", "i've", "if", "in", "into", "is", "it", "it's", "its", "itself", "let's", "me", "more", "most", "my", "myself", "nor", "of", "on", "once", "only", "or", "other", "ought", "our", "ours", "ourselves", "out", "over", "own", "same", "she", "she'd", "she'll", "she's", "should", "so", "some", "such", "than", "that", "that's", "the", "their", "theirs", "them", "themselves", "then", "there", "there's", "these", "they", "they'd", "they'll", "they're", "they've", "this", "those", "through", "to", "too", "under", "until", "up", "very", "was", "we", "we'd", "we'll", "we're", "we've", "were", "what", "what's", "when", "when's", "where", "where's", "which", "while", "who", "who's", "whom", "why", "why's", "with", "would", "you", "you'd", "you'll", "you're", "you've", "your", "yours", "yourself", "yourselves" ]

In [28]:
table = str.maketrans('', '', string.punctuation)

In [30]:
train_data = tfds.as_numpy(tfds.load('imdb_reviews', split="train"))

In [36]:
imdb_sentences = []
for item in train_data:
    sentence = str(item['text'].decode('UTF-8').lower())
    sentence = sentence.replace(",", " , ")
    sentence = sentence.replace(".", " . ")
    sentence = sentence.replace("-", " - ")
    sentence = sentence.replace("/", " / ")
    soup = BeautifulSoup(sentence)
    sentence = soup.get_text()
    words = sentence.split()
    filtered_sentence = ""
    for word in words:
        word = word.translate(table)
        if word not in stopwords:
            filtered_sentence = filtered_sentence + word + " "
    imdb_sentences.append(filtered_sentence)    

In [37]:
print(len(imdb_sentences))
print(imdb_sentences[0])

25000
absolutely terrible movie  dont lured christopher walken michael ironside  great actors  must simply worst role history  even great acting not redeem movies ridiculous storyline  movie early nineties us propaganda piece  pathetic scenes columbian rebels making cases revolutions  maria conchita alonso appeared phony  pseudo  love affair walken nothing pathetic emotional plug movie devoid real meaning  disappointed movies like  ruining actors like christopher walkens good name  barely sit  


In [38]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [39]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=25000)
tokenizer.fit_on_texts(imdb_sentences)
sequences = tokenizer.texts_to_sequences(imdb_sentences)

In [41]:
# print(tokenizer.word_index)

In [42]:
##'slajps': 142502, 'slajps': 86156

In [44]:
print(sequences[0])

[322, 290, 1, 24, 10403, 1264, 3472, 387, 10885, 22, 65, 118, 227, 151, 121, 371, 10, 22, 40, 3, 5550, 26, 537, 656, 1, 299, 7910, 86, 2345, 310, 1106, 54, 7324, 134, 2776, 17417, 2834, 20270, 18273, 1365, 4771, 3825, 42, 1463, 3472, 72, 1106, 806, 7775, 1, 4060, 59, 1095, 572, 26, 5, 6941, 65, 5, 1264, 13705, 7, 300, 1076, 753]


### decoding sequences

In [45]:
reverse_word_index = dict([(value, key) for (key, value) in tokenizer.word_index.items()])
decoded_review = ' '.join([reverse_word_index.get(i, '?') for i in sequences[0]])
print(decoded_review)

absolutely terrible movie dont lured christopher walken michael ironside great actors must simply worst role history even great acting not redeem movies ridiculous storyline movie early nineties us propaganda piece pathetic scenes rebels making cases revolutions maria conchita alonso appeared phony pseudo love affair walken nothing pathetic emotional plug movie devoid real meaning disappointed movies like ruining actors like christopher walkens good name barely sit


## IMDb subwords dataset

In [ ]:
(train_data, test_data), info = tfds.load(
                                'imdb_reviews/subwords8k',
                                split = (tfds.Split.TRAIN, tfds.Split.TEST),
                                as_supervised=True,
                                with_info=True
                                )

In [ ]:
encoder = info.features['text'].encoder
print ('Vocabulary size: {}'.format(encoder.vocab_size))

In [ ]:
print(encoder.subwords)

In [ ]:
sample_string = 'Today is a sunny day'
encoded_string = encoder.encode(sample_string)
print ('Encoded string is {}'.format(encoded_string))

In [ ]:
print(encoder.subwords[6426])

In [ ]:
encoded_string = encoder.encode(sample_string)
print(encoded_string)
original_string = encoder.decode(encoded_string)
print(original_string)
test_string = encoder.decode([6427, 4869, 9, 4, 2365, 1361, 606])
print(test_string)
test_string = encoder.decode(encoded_string)
print(test_string)